# Перезапускает ядро
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import random as r
from matplotlib.patches import Ellipse, Arc
import numpy as np

import math
import time
from time import gmtime, strftime, localtime
import os
import pandas as pd
import joblib
from joblib import Parallel, delayed, Memory
from multiprocessing import Queue, Manager
from threading import Thread
import itertools
from tqdm import tqdm

In [2]:
from ipynb.fs.full.Classes import *

In [3]:
class PrintThreadInfo():
    
    printQueue = Manager().Queue()
    columns = []
    fileName = "results.csv"
        
    def prepareCSV():
        PrintThreadInfo.columns = ["img_name", "attach_prob", "Kb", "N", "H", "D", "Vavg"]
        for i in range(CONSTS.ITER_COUNT // CONSTS.STEP_COUNT_GROW_VELOCITY):
            PrintThreadInfo.columns.append("v" + str(i))
        df = pd.DataFrame(columns = PrintThreadInfo.columns)
        df.to_csv(PrintThreadInfo.fileName, sep = ";", index = False)

In [4]:
def StartProcess(KB, N, H, D, queue, flag, iter_count):
    
    if flag == True:
        print("Queue started")
        
        for i in range(Settings.count()):
            info = PrintThreadInfo.printQueue.get()
            iter_c = info[4]
            print(f'{iter_c} element : get ')
            
            myTree = info[0]
            myCell = info[1]
            myConsts = info[2]
            
            plotName = "img/scr_" + str(myConsts) + ".png"
            
            myPlot = Plot([0, myCell.radius + 20], [-(myCell.radius + 20), myCell.radius + 20])
            myPlot.drawPlot(myTree, myCell, name = plotName, consts = myConsts)
            
            print(f'{iter_c} element : draw plot')

            listOfValues = [
                plotName, 
                myConsts.CREATE_BRANCH_PROB, 
                myConsts.Kb,
                myConsts.N_BRANCHES_TO_PUSH_SELL, 
                myConsts.DISTANCE_TO_GROW, 
                myConsts.DISTANCE_TO_ATTACH
            ]
            
            listOfValues.append(CONSTS.countVelocity(myCell.radius - myCell.startRadius, CONSTS.ITER_COUNT))
            
            for v in info[3]:
                listOfValues.append(v)

            df = pd.DataFrame(data = [listOfValues], columns = PrintThreadInfo.columns)
            df.to_csv(PrintThreadInfo.fileName, mode='a', sep = ";", header = None)
            
            print(f'{iter_c} element : write in .csv')
            
            PrintThreadInfo.printQueue.task_done()
            
            print(f'{iter_c} element : done')
            
    else:
        print(f'{iter_count} element : count started')
        
        myTree = Tree(Branch(0,0,Direction(0),None))
        myConsts = INNER_CONSTS(KB, N, H, D)
        myCell = Cell([0,0], myConsts.DISTANCE_TO_GROW)
        myRandomizer = Randomizer(myTree, myCell, myConsts)
        
        tempTime = time.time()
        for i in range(CONSTS.ITER_COUNT):
            if ((i+1) / CONSTS.ITER_COUNT) * 100 % 10 == 0:
                print(f'{iter_count} element : count {((i+1) / CONSTS.ITER_COUNT) * 100}% {round(time.time() - tempTime,1)}s')
                tempTime = time.time()
            myRandomizer.oneTick() 
            
        print(f'{iter_count} element : count done')
        queue.put([myTree, myCell, myConsts, myRandomizer.velocities, iter_count])
        print(f'{iter_count} element : put in Queue')

In [5]:
allCombinations = list(itertools.product(Settings.KB_RANGE, Settings.N_RANGE, Settings.H_RANGE, Settings.D_RANGE, [False]))
np.random.shuffle(allCombinations)
allCombinations.insert(0, (1,1,1,1,True))
allCombinations = [list(x) + [i] for i, x in enumerate(allCombinations)]
print(len(allCombinations))

4201


In [6]:
print(Settings.count())

4200


In [ ]:
PrintThreadInfo.prepareCSV()
Parallel(n_jobs=CONSTS.NUMBER_OF_THREADS)(delayed(StartProcess)(i[0], i[1], i[2], i[3], PrintThreadInfo.printQueue, i[4], i[5]) for i in allCombinations)

In [8]:
import pandas as pd
d = pd.read_csv("results.csv", sep = ";")
d

,img_name,attach_prob,Kb,N,H,D,Vavg,v0,v1,v2,...,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74
0,img/scr_1_2_4_4.png,9.699995e-07,1,2,4,4,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_2_2_5_5.png,1.939998e-06,2,2,5,5,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_2_2_5_4.png,1.939998e-06,2,2,5,4,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_1_2_5_5.png,9.699995e-07,1,2,5,5,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_1_2_5_4.png,9.699995e-07,1,2,5,4,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_2_2_4_5.png,1.939998e-06,2,2,4,5,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_1_2_4_5.png,9.699995e-07,1,2,4,5,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_1_1_4_4.png,9.699995e-07,1,1,4,4,2.72,3.02,2.75,2.75,...,3.58,3.30,3.02,2.48,3.30,2.75,3.30,3.58,1.38,3.30
0,img/scr_2_2_4_4.png,1.939998e-06,2,2,4,4,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0,img/scr_1_1_4_5.png,9.699995e-07,1,1,4,5,2.57,1.65,1.93,3.02,...,3.02,2.75,2.48,3.30,3.85,2.75,3.30,0.82,2.75,1.38


In [8]:
StartProcess(1,1,1,1,PrintThreadInfo.printQueue, False, 1 )

1 element : count started
1 element : count 10.0% 0.0s
1 element : count 20.0% 0.1s
1 element : count 30.0% 0.1s
1 element : count 40.0% 0.1s
1 element : count 50.0% 0.1s
1 element : count 60.0% 0.2s
1 element : count 70.0% 0.2s
1 element : count 80.0% 0.3s
1 element : count 90.0% 0.3s
1 element : count 100.0% 0.4s
1 element : count done
1 element : put in Queue
